In [41]:
import pybedtools #v0.8.1
from pybedtools import BedTool
import numpy as np
import pandas as pd  # v1.0.1
import argparse
import os

In [42]:
def ProcessRegion(RawGEMs):
    """
    Process the given region, retrive all valid GEMs
    """
    Temp = RawGEMs.groupby(g=[1,2,3,5], c=[5,6,7,8], o=['count','collapse','collapse','collapse'])
    RefineGEMs = Temp.filter(lambda F: int(F[4]) > 1)
    # Need this to keep the result of filter! All these files can be manually removed after the job
    Test = BedTool(RefineGEMs).saveas()
    Tempstr = ''
    for i in range(Test.count()):
        Start = np.fromstring(Test[i][6], dtype = np.int,  sep =', ' )
        End = np.fromstring(Test[i][7], dtype = np.int,  sep =', ' )
        Mcount = Test[i][5].count('P')
        Start.sort()
        End.sort()
        # chrom, start_min, end_min, GEM ID, #(fragments)  start_max, end_max,
        for j in range(len(Start)):
            if j == 0:
                Tempstr += Test[i][0]+' '+ str(Start[j]) + ' ' + str(End[j])+' '+ Test[i][3] + ' ' + str(len(Start)) + ' ' + str(Mcount)+' '
            elif len(Start)!=2 and j != (len(Start)-1):
                Tempstr += str(Start[j])+','+str(End[j])+','
            elif j == (len(Start)-1):
                Tempstr += str('-1,-1')+' ' + str(Start[j]) + ' ' + str(End[j]) + '\n'

    FinalGEMs = BedTool(Tempstr,from_string=True)
    return FinalGEMs

In [43]:
def Checkintersect(s1,s2,e1,e2):
    """
    Check for the left/right/both/none condition
    """
    return (min(e1, e2) - max(s1, s2)) > 0

In [44]:
def inInterval(FF,Temp,Type,Length,CHR):
    if CHR == FF[0]:
        interval = [0,0,0,0]
        interval[0] = Temp[0]-Length
        interval[1] = Temp[0]+Length+19
        interval[2] = Temp[1]-Length-19
        interval[3] = Temp[1]+Length

        NumFrag = FF[4]
        Start = list(map(int, FF[1:3]))
        End = list(map(int, FF[-2:]))
        if Type == 'Left':
            return (Checkintersect(interval[0],Start[0],interval[1],Start[1])) and not (Checkintersect(interval[2],End[0],interval[3],End[1]))
        elif Type == 'Right':
            return not (Checkintersect(interval[0],Start[0],interval[1],Start[1])) and (Checkintersect(interval[2],End[0],interval[3],End[1]))
        elif Type == 'Both':
            return (Checkintersect(interval[0],Start[0],interval[1],Start[1])) and (Checkintersect(interval[2],End[0],interval[3],End[1]))
        else:
            return not (Checkintersect(interval[0],Start[0],interval[1],Start[1])) and not (Checkintersect(interval[2],End[0],interval[3],End[1]))
    else:
        return False


In [45]:
def SortGEM(FinalGEMs, Region,Type,Length,savebedpath,library):
    """
    Classify FinalGEMs based on Type (left/right/both) in Region.
    left: start_min to start_min+Length; right: end_max-Length to end_max
    Also save the result automatically. Can change path.
    """
    Temp = list(map(int, Region[4:6]))
    CHR = Region[0]
    TypeGEMs = FinalGEMs.filter(inInterval, Temp,Type,Length,CHR).sort()#.saveas()
    # I use loop id to indicate region
#     savebedpath = 'Minji_data/Cohesin_results/01ALL/4kbext_dm/Bedfiles/'
    TypeGEMs = TypeGEMs.moveto(savebedpath + library + '_' + str(Region[3])+'_'+Type+'.bed')
    if Type == 'Both':
        Flag = 2
    elif Type == 'None':
        Flag = 0
    else:
        Flag = 1
    Count = 0
    Tot = TypeGEMs.count()
#     Check if any fragments intersect with middle motif
    for i in range(Tot):
        Mcount = int(TypeGEMs[i][5])
        if Mcount> Flag:
            Count = Count+1
    return Tot-Count, Count


In [46]:
def mainfunc(path1,path2,savebedpath,savecsvpath,tmpfilepath,RegInterval,Thread,Length,library):
    """
    path1: path for GEMs (i.e. ___ALL.region.PEanno)
    path2: path for Region (i.e. ____PETcnt_G9.motifannot)
    savebedpath: path for saving extracted GEMs in .bed
    savecsvpath: path for saving summary table in .csv
    tmpfilepath: path for saving tmpfiles produced by pybedtool, a directory
    Thread: for naming the csv file. (i.e. '0')
    Length: Length of extension. Default = 4000 (int)
    """
    pybedtools.helpers.cleanup()
    pybedtools.set_tempdir(tmpfilepath)
    # read in region files containing complexes (GEMs)
    ChIA_Drop = BedTool(path1)
    # read in domain or loop file
    Region = BedTool(path2)

    # Remove unnecessary entries
    Region_short = Region.groupby(g=[1,2,6,12,14,20,8,9,16,21], c=[12], o=['count'])
    Max_iter = Region_short.count()

    Dict = {}
    for i in RegInterval:
        # NowRegion: chrom, start_min, end_max, loop id, ...
        # This line can be improved...
        NowRegion = BedTool(Region_short[i:i+1]).saveas()
        # Find all fragments that intersect with Nowregion
        Intersection = ChIA_Drop.intersect(NowRegion,wa=True)
        # Append original start/and. Technical purpose for using groupby...
        results = [(f[0],'0','1',f[3],f[4],f[5],f[1],f[2]) for f in Intersection]
        IntersectionRaw = BedTool(results)

        # Sort the grouping key!!!! Otherwise the later groupby doesn't work as intended...
        Intersection = IntersectionRaw.sort(chrThenScoreA = True)
        # Extract the valid GEMs
        FinalGEMs = ProcessRegion(Intersection)
        # Classify+sort+save
        Count_L0,Count_L1 = SortGEM(FinalGEMs, NowRegion[0],'Left',Length,savebedpath,library)
        Count_R0,Count_R1 = SortGEM(FinalGEMs, NowRegion[0],'Right',Length,savebedpath,library)
        Count_B0,Count_B1 = SortGEM(FinalGEMs, NowRegion[0],'Both',Length,savebedpath,library)
        Count_N0,Count_N1 = SortGEM(FinalGEMs, NowRegion[0],'None',Length,savebedpath,library)
        Total = Count_L0+Count_L1+Count_R0+Count_R1+Count_B0+Count_B1+Count_N0+Count_N1

        # Write into dictionary
        Dict[NowRegion[0][3]] = [NowRegion[0][3],Count_L0,Count_L1,Count_L0+Count_L1,round((Count_L0+Count_L1)/Total*100, 2),
                                 Count_R0,Count_R1,Count_R0+Count_R1,round((Count_R0+Count_R1)/Total*100, 2),
                                 Count_B0,Count_B1,Count_B0+Count_B1,round((Count_B0+Count_B1)/Total*100, 2),
                                 Count_N0,Count_N1,Count_N0+Count_N1,round((Count_N0+Count_N1)/Total*100, 2),
                                 Total,Total-(Count_N0+Count_N1),round((Total-(Count_N0+Count_N1))/Total*100, 2),
                                 NowRegion[0][6],NowRegion[0][7],NowRegion[0][8],NowRegion[0][9],
                                 NowRegion[0][0]+':'+str(NowRegion[0][1])+'-'+str(NowRegion[0][2])]

    RenameCol = {}
    NewCol = ['LoopID','Left_0','Left_1','Left_Tol','Left_Tol %','Right_0','Right_1','Right_Tol','Right_Tol %',
              'Both_0','Both_1','Both_Tol','Both_Tol %',
              'None_0','None_1','None_Tol','None_Tol %','Total','Total-None','Total-None %',
              'Left Intensity', 'Right Intensity','Left motif strand', 'Right motif strand',
              'Region']
    for i, name in enumerate(NewCol):
        RenameCol[i] = NewCol[i]

    DF = pd.DataFrame.from_dict(Dict,orient = 'index').rename(columns = RenameCol)
    DF.to_csv(savecsvpath + library + '_' + 'LRBNstats_'+Thread+'.csv',index=False)
    # Clear all temp files for this session
    pybedtools.helpers.cleanup()

In [48]:
if __name__ == '__main__':
    """
    Developer: Eli Chien
    Maintainer: Minji Kim (questions addressed via minji.kim@jax.org)

    path1: path for GEMs (i.e. ___ALL.region.PEanno)
    path2: path for Region (i.e. ____PETcnt_G9.motifannot)
    savebedpath: path for saving extracted GEMs in .bed
    savecsvpath: path for saving summary table in .csv
    tmpfilepath: path for saving tmpfiles produced by pybedtool, a directory
    Thread: for naming the csv file. (i.e. '0')
    Length: Length of extension. Default = 4000 (int)
    """
    ## Example ##
    directory = '/Users/kimm/Desktop/Final_scripts_20200301/'
    library = 'testing'
    gemfile = 'CDH0002NR9L_hg38_CTCF_filt_comp_FDR_0.1_ALL_motifext4kbboth.region.PEanno'
    domainfile = 'LHG0052H.e500.clusters.cis.bothanchint_G250.PETcnt_G9.motifannot.sorted.domains'
    #savebedpath = '/Users/kimm/Desktop/chiadrop_scripts/output/'
    #savecsvpath = '/Users/kimm/Desktop/chiadrop_scripts/output/'
    #tmpfilepath = '/Users/kimm/Desktop/chiadrop_scripts/tmp'
    Start_pos = 0
    End_pos = 10
    Thread = '0'
    Length = 4000

#     parser = argparse.ArgumentParser()
#     parser.add_argument('--dir',type = str) # main directory
#     parser.add_argument('--lib', type = str) # library name
#     parser.add_argument('--GEMfile',type = str) # name of GEM file in *region.PEanno format
#     parser.add_argument('--dmfile',type = str) # name of domain or loop file in *motifannot.sorted.domains format
#     parser.add_argument('--Start_pos',type = int)
#     parser.add_argument('--End_pos',type = int)
#     parser.add_argument('--Length',type = int)
#     parser.add_argument('--Thread',type = str)
#     args = parser.parse_args()

#     directory = args.dir
#     gemfile = args.GEMfile
#     domainfile = args.dmfile
#     library = args.lib
#     Start_pos = args.Start_pos
#     End_pos = args.End_pos
#     Thread = str(args.Thread)
#     Length = args.Length

    path1 = directory + gemfile
    path2 = directory + domainfile
    savebedpath = directory + library + "/" + "bedfile/"
    savecsvpath = directory + library + "/" + "tables/"
    tmpfilepath = directory + library + "/" + "tmp_" + Thread
    if not os.path.exists(directory + library):
        os.mkdir(directory + library)
    if not os.path.exists(savebedpath):
        os.mkdir(savebedpath)
    if not os.path.exists(savecsvpath):
        os.mkdir(savecsvpath)
    if not os.path.exists(tmpfilepath):
        os.mkdir(tmpfilepath)

    RegInterval = range(Start_pos,End_pos)
    # Note that it is 0-based.
    # It will process through exactly Start_pos to End_pos-1 (i.e. range(Start_pos,End_pos))

    mainfunc(path1,path2,savebedpath,savecsvpath,tmpfilepath,RegInterval,Thread,Length,library)
